<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/algorithm/NonLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 非線形方程式の数値解法

3次方程式
\begin{equation}
x^3+6x^2+21x+32=0
\end{equation}
の解は
\begin{equation}
x=-9^{1/3}+3^{1/3}-2,\ -9^{1/3}e^{i2\pi/3}+3^{1/3}e^{i4\pi/3}-2,\ -9^{1/3}e^{i4\pi/3}+3^{1/3}e^{i2\pi/3}-2
\end{equation}
または
\begin{equation}
x\sim-2.64,\ -1.68-3.05i,\ -1.68+3.05i
\end{equation}

In [ ]:
{}